In [1]:
# This method is to extract the hierarchy relationship between concepts from OMOP, then track back the concepts' ancestors
# to see that whether they are similar (by sharing common ancestor)

# The results is not consistent with the opinions from domain expert probably because the way OMOP store the hierarchy relationship
# information is either too general or too detailed

# To excute the code below, SQL Server is required, and local database can be built by downloeading from http://athena.ohdsi.org/

In [1]:
import numpy as np
import pandas as pd

In [4]:
crit = pd.read_csv('all_criteria_for_azure.csv')

In [6]:
import pyodbc

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-0PE7IP3\SQLEXPRESS01;'
                      'Database=OMOP;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

In [7]:
def get_ancestor(concept_id, level):
    
    query = \
    f'''select ancestor_concept_id
    from CONCEPT_ANCESTOR
    where descendant_concept_id = {concept_id} and min_levels_of_separation < {level}'''
    
    cursor.execute(query)

    return set([ancestor_id[0] for ancestor_id in cursor])

In [8]:
concept_name_list = ['Disease caused by severe acute respiratory syndrome coronavirus 2',
 'Patient currently pregnant',
 'Human coronavirus',
 'Blood oxygen saturation',
 'Fever',
 'Aspartate aminotransferase measurement',
 'Artificial respiration',
 'Allergy to hydroxychloroquine',
 'Dyspnea',
 'Cough',
 'Alanine aminotransferase measurement',
 'Oxygen therapy',
 'Retinal disorder',
 'Hydroxychloroquine',
 'Chronic kidney disease',
 'Estimation of glomerular filtration rate',
 'Dialysis procedure',
 'Glucose-6-phosphate dehydrogenase deficiency anemia',
 'Allergy to study medication',
 'IMMUNOSUPPRESSANTS',
 'Q-T interval corrected',
 'Disease of liver',
 'Human immunodeficiency virus infection',
 'Respiratory rate',
 'Measurement of renal clearance of creatinine',
 'Prolonged QT interval',
 'Cirrhosis of liver',
 'Porphyria',
 'Ratio of arterial oxygen tension to inspired oxygen fraction',
 'Mycosis']

concept_id_list = [crit[crit['concept_name']==i]['concept_id'].values[0] for i in concept_name_list]

In [10]:
ancestors_list = [get_ancestor(c_id, 5) for c_id in concept_id_list]

In [148]:
groups = {key: None for key in range(len(ancestors_list))}
pools = dict()

for i in range(len(ancestors_list)):
    for group, concept_ids in pools.items():
        if len(ancestors_list[i] & concept_ids) > 0:
            groups[i] = group
            pools[group] = (ancestors_list[i] | concept_ids)
    if groups[i] == None: 
        groups[i] = len(pools)
        pools[len(pools)] = ancestors_list[i]

In [160]:
for i in range(len(pools)):
    print('')
    print(f'//////////////GROUP{i}////////////////////')
    for key, value in groups.items():
        if value == i:
            print(concept_name_list[key])


//////////////GROUP0////////////////////
Disease caused by severe acute respiratory syndrome coronavirus 2
Human immunodeficiency virus infection

//////////////GROUP1////////////////////
Patient currently pregnant

//////////////GROUP2////////////////////
Human coronavirus

//////////////GROUP3////////////////////
Blood oxygen saturation
Aspartate aminotransferase measurement
Alanine aminotransferase measurement
Estimation of glomerular filtration rate

//////////////GROUP4////////////////////
Fever

//////////////GROUP5////////////////////
Artificial respiration
Oxygen therapy
Dialysis procedure
Measurement of renal clearance of creatinine
Ratio of arterial oxygen tension to inspired oxygen fraction

//////////////GROUP6////////////////////
Allergy to hydroxychloroquine

//////////////GROUP7////////////////////
Dyspnea
Cough
Chronic kidney disease
Glucose-6-phosphate dehydrogenase deficiency anemia
Disease of liver
Prolonged QT interval
Cirrhosis of liver
Porphyria
Mycosis

////////